## ToDos:
- alternate who starts the game... (make it more robus)

In [1]:
import itertools 
import random
random.seed(10)
from tictactoe import *
import numpy as np
import math

In [2]:
# hyper-params
alpha = 0.1
gamma = 0.2
n_episodes = 100

EMPTY_FIELD = 0
OUR_AGENT = 1
ENEMY_OPPONENT = 2

In [3]:
def _flatten(nested):
    flat = []
    for l in nested:
        flat.extend(l)
    else:
        return flat

In [4]:
def init_lut():
    """
    0 ... empty fields
    1 ... our AGENT's prior moves
    2 ... OPPONENT's moves
    """
    global lut
    lut = {}
    for state in itertools.product([0,1,2],repeat=9): 
        lut[state] = [round(random.uniform(0, 1),1) if s == 0 else float('nan') for s in state]
        
        # check for terminal state
        board = [state[:3],state[3:6],state[6:9]]
        if has_won(board, 1) or has_won(board, 2):
            lut[state] = [0.0 if s == 0 else float('nan') for s in state]
init_lut()
# lut

In [5]:
def select_action(board):
    q_i = int(np.nanargmax(lut[tuple(_flatten(board))]))
    return (math.floor(q_i / 3), q_i % 3)

# select_action(board)

In [ ]:
board = init_board()

In [ ]:
choose_action(board)

In [ ]:
# initialize lut
init_lut()

for x in range(n_episodes):
    init_board()
    
    # print(x)

In [ ]:
from tictactoe import *

In [ ]:
board = init_board()

In [ ]:
board

In [ ]:
print_board(board)

In [ ]:
x = get_move(board, 0)

In [ ]:
mark(board, 1, *x)

In [ ]:
x

In [ ]:
get_ai_move(board, 1)

## Test Run Through
- afterwards check if maybe you misunderstood something? how did he implement it?

In [ ]:
board = init_board()

In [ ]:
board

In [ ]:
np.nanargmax(lut[tuple(_flatten(board))])

In [ ]:
lut[tuple(_flatten(board))]

In [ ]:
# player 1 (that is us) always starts
# WE MAKE A MOVE
a = select_action(board)
mark(board, 1, *a)
print_board(board)

In [ ]:
a = get_ai_move(board, 2)
mark(board, 2, *a)
print_board(board)

In [ ]:
# THEY MAKE A MOVE


In [ ]:
lut[tuple(_flatten(board))]

In [17]:
agent_wins = 0
opponents_wins = 0
draw = 0

for _ in range(n_episodes):
    board = init_board()
    while not has_won(board, 1) and not has_won(board, 2) and not is_full(board):
        agent_move = select_action(board)
        mark(board, 1, *agent_move)
        if not has_won(board, 1) and not is_full(board):
            row, col = get_ai_move(board, 2)
            mark(board, 2, row, col)

    if has_won(board, 1):
        agent_wins += 1
    elif has_won(board, 2):
        opponents_wins += 1
    else:
        draw += 1


print(f'agent_wins: {agent_wins}')
print(f'opponents_wins: {opponents_wins}')
print(f'draw: {draw}')

agent_wins: 0
opponents_wins: 92
draw: 8
